In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/dataset.csv')

In [ ]:
len(df)

18634

In [ ]:
df.head(2)

,Unnamed: 0,Email Text,Email Type,Reason,Recommendation,label
0,0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email,The email content does not raise any red flags...,No special action is required. Handle this ema...,0
1,1,the other side of * galicismos * * galicismo *...,Safe Email,The email content does not raise any red flags...,No special action is required. Handle this ema...,0


In [1]:
# pip install torch==1.12.0 torchtext==0.13.0

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer
import re
import torch
from nltk.corpus import stopwords, words as nltk_words
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer
from sklearn.model_selection import train_test_split
from nltk.corpus import words
# Download necessary NLTK corpora
import nltk
nltk.download('stopwords')
nltk.download('words')
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

### Data cleaning

In [ ]:
def preprocess_string(s):
    s = re.sub(r"[^\w\s]", '', s)
    s = re.sub(r"\s+", ' ', s)     # Replace multiple spaces with a single space
    s = re.sub(r"\d", '', s)       # Remove digits
    return s.strip()


english_words = set(words.words())
stop_words = set(stopwords.words('english'))

def remove_stop_words(text):
  
    tokens = word_tokenize(text) 
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)


def remove_non_english_words(text):
    words_in_text = re.findall(r'\b\w+\b', text)
    english_only = [word for word in words_in_text if word.lower() in english_words]
    return ' '.join(english_only)

In [ ]:
df['cleaned_text'] = df['Email Text'].apply(preprocess_string)
df['cleaned_text'] = df['cleaned_text'].apply(remove_non_english_words)
df['cleaned_text'] = df['cleaned_text'].apply(remove_stop_words)
email_content = df['cleaned_text'].tolist()
label = df['label'].tolist()

In [ ]:
email_content[0:5]

['re disc uniformitarianism re sex dick s on us use of s on but not d as a vocative are very but i am not sure that it is fair to attribute this to sons being like senior for one thing we do normally use brother in this way any more than we do d and it is hard to imagine a natural class senior and s on but excluding brother for another there seem to me to be here if i am not a distinction that is not there it to me that the senior relative are used in a variety of e g calling out from a distance to get someone s attention and hence at the beginning of an utterance whereas s on more natural in like yes son hand me that son than in like son or son help me although perhaps these latter are not completely impossible',
 'the other side of is a term which the improper introduction of which are sounding and thus very deceptive to the ear is often considered to be a what would be the term which the opposite phenomenon that is unlawful of origin which may have crept into can someone provide tha

## Deciding sequence length

In [ ]:
from collections import Counter
from nltk.tokenize import word_tokenize

# create a custom dataset class
class TextDataset():
    def __init__(self, sentences):
        self.sentences = sentences
        self.word2idx = {}
        self.idx2word = {}

        # create vocabulary
        word_freq = Counter()
        for sent in self.sentences:
            #tokens consists of words in a sentence, e.g., ['think', 'of', 'the', 'film', 'being', 'like', 'a', 'dream', '.']
            tokens = word_tokenize(sent.lower()) # convert to lowercase and tokenize
            word_freq.update(tokens)

        # vocabulary sorted by frequency (most common words first)
        vocabulary = [word for word, freq in word_freq.most_common()]

        self.word2idx = {word: idx for idx, word in enumerate(vocabulary)}
        self.idx2word = {idx: word for word, idx in self.word2idx.items()}

In [ ]:
dataset = TextDataset(email_content)

word2idx = dataset.word2idx
idx2word = dataset.idx2word

In [ ]:
print(list(word2idx.items())[0:10])

[('university', 0), ('language', 1), ('one', 2), ('information', 3), ('please', 4), ('new', 5), ('would', 6), ('subject', 7), ('e', 8), ('may', 9)]


In [ ]:
#

seq_lengths = np.array([len(s.split()) for s in email_content])
print([(p, np.percentile(seq_lengths, p)) for p in [75, 80, 90, 95, 99, 100]])

[(75, 101.0), (80, 125.0), (90, 210.0), (95, 349.0), (99, 878.6699999999983), (100, 901935.0)]


So I choose 350 as the sequence legnth

## Create vocab

In [ ]:
from collections import Counter

# tokenizer function using split
def simple_tokenizer(text):
    return word_tokenize(text.lower())

# generator function to yield tokens
def yield_tokens(data_iter):
    for text in data_iter:
        yield simple_tokenizer(text)

# build vocabulary using Counter and add special tokens
def build_vocab_from_iterator(iterator, specials=None, min_freq=3):
    word_freq = Counter()

    # Count the frequencies of the tokens in the iterator
    for tokens in iterator:
        word_freq.update(tokens)

    # Filter out words that appear less than or equal to 3 times
    filtered_word_freq = {word: freq for word, freq in word_freq.items() if freq > min_freq}

    # Sort words by frequency (most common first) and include special tokens
    sorted_vocab = specials + [word for word, freq in sorted(filtered_word_freq.items(), key=lambda x: x[1], reverse=True)]

    # Create a vocabulary with indices
    vocab = {word: idx for idx, word in enumerate(sorted_vocab)}

    return vocab

# build the vocabulary
vocab = build_vocab_from_iterator(yield_tokens(email_content), specials=["<pad>"])

# print the vocabulary and default index
print("Vocabulary:", vocab)

Vocabulary: {'<pad>': 0, 'university': 1, 'language': 2, 'one': 3, 'information': 4, 'please': 5, 'new': 6, 'would': 7, 'subject': 8, 'e': 9, 'may': 10, 'also': 11, 'get': 12, 'time': 13, 'us': 14, 'de': 15, 'like': 16, 'linguistics': 17, 'list': 18, 'company': 19, 'address': 20, 'mail': 21, 'people': 22, 'conference': 23, 'free': 24, 'use': 25, 'business': 26, 'make': 27, 'order': 28, 'work': 29, 'send': 30, 'first': 31, 'know': 32, 'report': 33, 'name': 34, 'program': 35, 'money': 36, 'research': 37, 'two': 38, 'call': 39, 'c': 40, 'could': 41, 'message': 42, 'see': 43, 'said': 44, 'need': 45, 'number': 46, 'many': 47, 'p': 48, 'available': 49, 'l': 50, 'web': 51, 'following': 52, 'th': 53, 'well': 54, 'way': 55, 'sent': 56, 'u': 57, 'contact': 58, 'want': 59, 'j': 60, 'workshop': 61, 'system': 62, 'take': 63, 'day': 64, 'price': 65, 'within': 66, 'much': 67, 'even': 68, 'data': 69, 'b': 70, 'paper': 71, 'year': 72, 'world': 73, 'site': 74, 'group': 75, 'r': 76, 'form': 77, 'best': 7

In [ ]:
len(vocab)

18372

## Tokenize and process the text data

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

max_sequence_length = 350
vocab["<unk>"] = len(vocab)
unk_index = vocab["<unk>"]


# Tokenize sentences using word_tokenize
tokenized_sentences = [word_tokenize(email.lower()) for email in email_content]

# Transform sentences to sequences of indices, mapping unknown words to <unk>
sentences_as_ints = [[vocab.get(token, unk_index) for token in tokens] for tokens in tokenized_sentences]

# Pad and truncate sequences
padded_sequences = []
for seq in sentences_as_ints:
    if len(seq) > max_sequence_length:
        seq = seq[:max_sequence_length]

    # Pad sequence to max_sequence_length
    padding_length = max_sequence_length - len(seq)
    if padding_length > 0:
        seq.extend([0] * padding_length)  # pad with 0 (assuming 0 is the padding index)

    padded_sequences.append(seq)

# Convert list of lists to tensor
truncated_padded_sequences = torch.tensor(padded_sequences, dtype=torch.long)

# Example labels (replace with your actual labels)
labels_as_ints = torch.tensor(label)

# Create dataset
dataset = TensorDataset(truncated_padded_sequences, labels_as_ints)


In [ ]:
first_sequence, first_label = dataset[0]

# Print the sequence and label
print("First sequence (padded):", first_sequence)
print("First label:", first_label)

First sequence (padded): tensor([ 2421,  8371,   680,  2451,    14,    25, 10092,   198,  1933,  4063,
         5150,    16,   969,     3,   395,  1819,    25,  2775,    55,   329,
         1379,   157,   606,   969,  2896,  2775,   181,   682,  1386,   969,
         1235,   103,   816,     9,   106,  1461,  1550,    12,   386,   631,
         1636,   868,  3109,  1953,   157,    16,   461,  1637,   611,  1637,
           16,  1637,  1637,   102,   512,   655,  1920,  1021,  1661,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,    

## Define and training the model

**Deeper LSTM with drop out**

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

class LSTMDropoutClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_size, num_layers, dropout):
        super(LSTMDropoutClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)  # Embedding layer

        # First LSTM layer
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim, num_layers=1, batch_first=True)
        self.dropout1 = nn.Dropout(dropout)  # Dropout after first LSTM

        # Second LSTM layer
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, num_layers=1, batch_first=True)
        self.dropout2 = nn.Dropout(dropout)  # Dropout after second LSTM

        # Fully connected layer
        self.fc1 = nn.Linear(hidden_dim, 64)  # First dense layer
        self.relu = nn.ReLU()  # ReLU activation

        # Final classification layer
        self.fc2 = nn.Linear(64, output_size)  # Final output layer with 24 classes

    def forward(self, x):
        # Embedding
        x = self.embedding(x)

        # First LSTM layer
        lstm_out, _ = self.lstm1(x)
        lstm_out = self.dropout1(lstm_out)

        # Second LSTM layer
        lstm_out, _ = self.lstm2(lstm_out)
        lstm_out = self.dropout2(lstm_out)

        # Get the final hidden state 
        final_hidden_state = lstm_out[:, -1, :]  # Get the last output of the LSTM

        # Pass through dense layers
        out = self.fc1(final_hidden_state)
        out = self.relu(out)
        out = self.fc2(out)

        return out

In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim

# Hyperparameters
BATCH_SIZE = 32
embedding_dim = 128  # Embedding size
hidden_dim = 256  # Hidden size for LSTM
output_size = 24  # Number of output classes 
num_layers = 2  # Number of LSTM layers
dropout = 0.2  # Dropout rate
vocab_size = len(vocab)+1  
num_epochs = 10
learning_rate = 0.001

train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Initialize the model
model = LSTMDropoutClassifier(vocab_size, embedding_dim, hidden_dim, output_size, num_layers, dropout)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(num_epochs):
    model.train()  
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()  
        outputs = model(sequences)  
        loss = criterion(outputs, labels)  
        loss.backward()  
        optimizer.step()  

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

        running_loss += loss.item()

    # Epoch loss and accuracy
    epoch_loss = running_loss / len(train_loader)
    accuracy = correct_predictions / total_predictions * 100

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

Epoch 1/10, Loss: 1.7486, Accuracy: 46.46%
Epoch 2/10, Loss: 1.7168, Accuracy: 46.88%
Epoch 3/10, Loss: 1.7142, Accuracy: 46.88%
Epoch 4/10, Loss: 1.7137, Accuracy: 46.88%
Epoch 5/10, Loss: 1.7235, Accuracy: 46.76%
Epoch 6/10, Loss: 1.7087, Accuracy: 46.89%
Epoch 7/10, Loss: 1.6240, Accuracy: 48.63%
Epoch 8/10, Loss: 1.3092, Accuracy: 58.62%
Epoch 9/10, Loss: 0.9641, Accuracy: 67.82%
Epoch 10/10, Loss: 0.7117, Accuracy: 75.47%


In [ ]:
for epoch in range(num_epochs):
    model.train()  
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()  
        outputs = model(sequences)  
        loss = criterion(outputs, labels)  
        loss.backward()  
        optimizer.step()  

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

        running_loss += loss.item()

    # Epoch loss and accuracy
    epoch_loss = running_loss / len(train_loader)
    accuracy = correct_predictions / total_predictions * 100

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

Epoch 1/10, Loss: 0.5980, Accuracy: 79.05%
Epoch 2/10, Loss: 0.5106, Accuracy: 82.22%
Epoch 3/10, Loss: 0.4348, Accuracy: 85.33%
Epoch 4/10, Loss: 0.3717, Accuracy: 87.11%
Epoch 5/10, Loss: 0.3201, Accuracy: 89.07%
Epoch 6/10, Loss: 0.2705, Accuracy: 90.46%
Epoch 7/10, Loss: 0.2453, Accuracy: 91.30%
Epoch 8/10, Loss: 0.2161, Accuracy: 92.16%
Epoch 9/10, Loss: 0.1901, Accuracy: 93.17%
Epoch 10/10, Loss: 0.1615, Accuracy: 94.01%


save the model to local machine

In [ ]:
torch.save(model.state_dict(), "deeper_bilstm_with_dropout.pth")

Load the model

In [ ]:
BATCH_SIZE = 32
embedding_dim = 128  
hidden_dim = 256 
output_size = 24  
num_layers = 2 
dropout = 0.2 
vocab_size = len(vocab) + 1 

In [ ]:
loaded_model = LSTMDropoutClassifier(vocab_size, embedding_dim, hidden_dim, output_size, num_layers, dropout)


loaded_model.load_state_dict(torch.load("deeper_bilstm_with_dropout.pth"))
loaded_model.eval()

LSTMDropoutClassifier(
  (embedding): Embedding(18374, 128, padding_idx=0)
  (lstm1): LSTM(128, 256, batch_first=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (lstm2): LSTM(256, 256, batch_first=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=24, bias=True)
)

## Test the model using new dataset

Text processing for user input

In [ ]:
def process_single_text(text, vocab, max_sequence_length=350, unk_index=None):

    # Preprocess the input text (cleaning, removing non-English words, and stopwords)
    text = preprocess_string(text)
    text = remove_non_english_words(text)
    text = remove_stop_words(text)

    # Tokenize the preprocessed text
    tokens = word_tokenize(text.lower())

    # Convert tokens to their corresponding indices in the vocab, using <unk> for unknown words
    if unk_index is None:
        unk_index = vocab.get("<unk>", len(vocab))  

    token_indices = [vocab.get(token, unk_index) for token in tokens]

    # Pad or truncate the tokenized sequence
    if len(token_indices) > max_sequence_length:
        token_indices = token_indices[:max_sequence_length]  
    else:
       
        token_indices.extend([0] * (max_sequence_length - len(token_indices)))

    # Convert the sequence to a tensor
    processed_tensor = torch.tensor(token_indices, dtype=torch.long)

    return processed_tensor

In [ ]:
#Phishing
single_email = "DEAR SIR/MADAM,MY NAME IS ENGINEER. TONY EDEM THE DIRECTOR GENERAL WITH THE MINISTRY OF WO=RKS AND HOUSING HERE IN NIGERIA. I WISH TO EXTENT THIS BUSINESS RELATIONSHI=P TO YOU WITH THE HOPE THAT YOU WILL ACCEPT IT. IN THE YEAR 1999, MY MINIST=RY AWARDED A CONTRACT TO A FOREIGN FIRM FOR BUILDING OF 620 HOUSING ESTATE =AND CONSTRUCTION OF A STADIUM IN ABUJA THE NEW CAPITAL TERRITERY MEANT FOR =THE CONCLUDED ALL AFRICAN GAMES WHICH NIGERIA HAVE HOSTED BY OCTOBER THIS Y=EAR.DURING THE TIME OF AWARDING THIS CONTRACT, MYSELF AND SOME TOP MEMBERS OF C=ONTRACT AWARDING COMMITTEE AGREED AND OVERINVOICED THE VOLUM OF THE CONTRAC=T WITH ADDITIONAL SUM OF $21 MILLION U.S.A DOLLARS, WHICH WAS MEANT TO TAKE= CARE OF US.AS AT LAST YEAR, THE CONTRACTOR HAD COMPLETED THE PROJECT AND F=ULLY PAID.FOR NOW THE OVERINVOICED SUM OF $21 MILLION U.S. DOLLARS IS WITH THE CENTRA=L BANK OF NIGERIA. AND MYSELF AND PARTNERS HAVE ALSO COMPLETED ARRANGMENT W=ITH TOP OFFICIAL OF THE BANK ON HOW TO RELEASE THIS MONEY, BUT FOR THE FACT= THAT THE ORIGINAL CONTRACTOR WAS A FOREIGNER WE ARE NOW LOOKING FOR A FORE=IGN PARTNER WHO WILL ASSIST US BY GIVING US ANY BANK ACCOUNT OF HIS/HER CHO=ICE TO ENABLE US ACCOMPLESH THIS BUSINESS.BASED ON THIS IAM NOW SOLICITING FOR YOUR KIND HELP TO GET THIS MUTUAL BUSI=NESS GO THROUGH.WHAT I NEED FROM YOU IS AN ACCOUNT WHERE THIS MONEY WILL BE= TRANSFERED INTO. EVERY ARRANGMENT HAVE BEEN PUT IN PLACE TO MAKE SURE THE =BUSINESS RUNS SMOOTHLY. TO THIS END, THE BUSINESS IS 100% RISK FREE. IN YOU=R ACCEPTANCEREPLY, WE SHALL NEGOTIATE THE PERCENTAGE THAT WILL BE GIVEN TO YOU FOR ASSI=STING US.I LOOK FORWARD FOR YOUR KIND AND IMMEDIATE RESPONSE."

processed_email_tensor = process_single_text(single_email, vocab, max_sequence_length=350, unk_index=vocab["<unk>"])



processed_email_tensor = processed_email_tensor.unsqueeze(0) 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processed_email_tensor = processed_email_tensor.to(device)


model = loaded_model.to(device)


model.eval()


with torch.no_grad():  
    logits = model(processed_email_tensor)

# 7. Apply softmax to get probabilities 
probabilities = torch.softmax(logits, dim=1)

label = torch.argmax(probabilities, dim=1).item()
label

Logits: tensor([[ -3.2872,   5.2870,   0.9525,  -4.7124, -16.3209,  -1.3826, -13.2843,
          -2.5193,  -6.4532,  -4.0646,  -5.5907,  -1.7851,  -4.3743, -10.1795,
         -12.6636,  -4.4622, -10.5178,  -9.8651,  -8.1280, -17.5557,  -4.5898,
         -11.3433, -16.7982, -12.5192]])
Probabilities: tensor([[1.8591e-04, 9.8410e-01, 1.2900e-02, 4.4702e-05, 4.0627e-10, 1.2486e-03,
         8.4647e-09, 4.0066e-04, 7.8398e-06, 8.5438e-05, 1.8574e-05, 8.3489e-04,
         6.2684e-05, 1.8880e-07, 1.5746e-08, 5.7412e-05, 1.3460e-07, 2.5855e-07,
         1.4687e-06, 1.1818e-10, 5.0534e-05, 5.8964e-08, 2.5209e-10, 1.8192e-08]])


In [ ]:
#Phising

single_email = "Dear Sir or Madam:We are keenly interested in setting up a new factory to produce 120,000 ERW oil casings per year to meet the big market of Shengli Oilfied. Since we lack some starting-up capital, we are seeking for investments. I will give you a brief introduction to the project as follow:I. Background:Shengli Oilfield, the second largest oilfield in China and the leading company of Sinopec Corp. (http://www.sinopec.com), as well as neighboring oilfields as Dagang Oilfield, Central Plain Oilfield and Jiangsu Oilfield are potential market of ERW oil casing. It is expected that the total oil casing needed in the above area is about 400,000 tons, including 260,000 tons in Sinopec and 150,000 tons in Shengli Oilfield. To meet the great need, we plan to set up a joint venture to produce 120,000 tons of ERW casings per year in Shengli Oilfield.  With good management and market outlets, we believe that this JV also has the potential to expand beyond China to other nations. II. About usShengli Oilfield Engineering Machinery Factory is one of sub-companies of Shengli Oilfield Co., Ltd, with total assets of 267 million yuan, net assets of 170 million yuan and 2462 staff. The factory mainly produces oilfield special equipment such as beam unit, Spiral-seam submerged-arc weldedsteel pipe, sucker rod, line pipe, casing, tubing, plastic compound pipe etc. It produces 200,000 tons of line pipes, 1 million meters of strengthened plastic compound pipes, 500 beam units, 2.5 million meters of sucker rods, 12,000 tons of PP gas pipes and 50,000 tons of oil tubings yearly, and has become important pipe base in Shandong and Shengli Oilfield. Its products have entering many areas as Shengli Oilfield, Daqing Oilfield, Liaohe Oilfield, Dagang Oilfield and so on. In 2002, the sale income amounted to over 600 million yuan.The Factory holds Quality accreditations as ISO9001, API and so on.III. Content of the projectIt is planned to introduce Japanese technology and equipment to set up one production line with range of casing diameter fromï¿½ï¿½114 to ï¿½ï¿½339.7 and produce 120,000 tons of ERW casings and delivery pipes per year.IV. Product specificationsFor Casing: grade of steel are J55 and N80, specification ranges from ï¿½ï¿½114 to ï¿½ï¿½339.7For delivery: grade of steel are X52ï¿½ï¿½X60ï¿½ï¿½X70, specification ranges from ï¿½ï¿½114 to ï¿½ï¿½339.7.V. Economic AnalysisThe total investment in this project is 142.41 million yuan. After the completion of the project, it is estimated that the internal financial rate of return before paying tax is 24.9 percent with a payback period of 5.6 years, and the internal rate of return after paying tax is 21.8 percent with a payback period of 5.9 years.The break-even point of the project is 46.95 percent, which means the project can keep balance if the annual output amount to 42,000 tons.VI. Plans for cooperationWe have set up a joint venture with Japan's Marubeni in the early of the year, with a registered capital of 57 million yuan. The capital structure is as follow:a. Shengli Oilfield Bureau of Administration          19.95 million yuan   35ï¿½ï¿½b. Shengli Oilfield Engineering Machinery Facotry     17.10 million yuan   30ï¿½ï¿½c. Dongying District State Assets Operation & Management Co., Ltd 5.70million yuan 10ï¿½ï¿½d. Marubeni Engineering Technology Co., Ltd           14.25 million yuan   25ï¿½ï¿½To meet our capital need for this project, we are now planned to increase our registered capital to 118.88 million yuan. Taking our added investment into consideration, we need another 34.68 million yuan to get the plan executed. So we are seeking a new joint venture partner who'd like to join this project and meet our capital requirement.We would like to exchange the information with the manufacturers and suppliers who are interested in this project and have discussion on raw material supply as well as jointly investing the factory establishment. And we hope we could have good cooperation in future.For additional information regarding this project, please feel free to contact us.Looking forward to your prompt reply.Sincerely yours, Tony LeungProject officer"

# Process the single email
processed_email_tensor = process_single_text(single_email, vocab, max_sequence_length=350, unk_index=vocab["<unk>"])



processed_email_tensor = processed_email_tensor.unsqueeze(0) 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processed_email_tensor = processed_email_tensor.to(device)


model = loaded_model.to(device)


model.eval()


with torch.no_grad():  
    logits = model(processed_email_tensor)

# 7. Apply softmax to get probabilities 
probabilities = torch.softmax(logits, dim=1)

label = torch.argmax(probabilities, dim=1).item()
print(label)

1


In [ ]:
single_email = "THIS IS AN OFFICIAL NOTIFICATION OF FUNDS DEPOSITED IN YOUR NAME AND IT IS N=OTA SALES SOLICITATION.I am professor Charles.c. Soludo governor, central bank of Nigeria ( =20CBN ). Thementioned bank is known for  the safekeeping of special and valuable"

# Process the single email
processed_email_tensor = process_single_text(single_email, vocab, max_sequence_length=350, unk_index=vocab["<unk>"])



processed_email_tensor = processed_email_tensor.unsqueeze(0)  


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processed_email_tensor = processed_email_tensor.to(device)


model = loaded_model.to(device)


model.eval()

# 6. Make a prediction
with torch.no_grad():  
    logits = model(processed_email_tensor)

# 7. Apply softmax to get probabilities 
probabilities = torch.softmax(logits, dim=1)

label = torch.argmax(probabilities, dim=1).item()
print(label)

1


In [ ]:
single_email = "DEAR SIR.I AM A GIRL OF 25  YEARS AND I AM LOOKING FOR INVESTMENT OPPORTUNITY.I INTENDED TO INVEST THE SUM OF TWENTY MILLION UNITED STATES DOLLARS  =INHERITED BY MY LATE FATHER.I AM FROM ZIMBABWE.BUT I AM LIVING IN THE =NETHERLANDS (EUROPE) AT THE MOMENT FOR MORE INFORMATION REACH ME  THROUGH =THIS  EMAIL    zuka2004@excite.com  BEST REGARDSMS.MUDIWA  ZUKA   --bff27629-3fab-43a9-b5ea-6602017a954c--"

# Process the single email
processed_email_tensor = process_single_text(single_email, vocab, max_sequence_length=350, unk_index=vocab["<unk>"])



processed_email_tensor = processed_email_tensor.unsqueeze(0) 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processed_email_tensor = processed_email_tensor.to(device)


model = loaded_model.to(device)


model.eval()

# 6. Make a prediction
with torch.no_grad():  
    logits = model(processed_email_tensor)

# 7. Apply softmax to get probabilities 
probabilities = torch.softmax(logits, dim=1)

label = torch.argmax(probabilities, dim=1).item()
print(label)

1


In [ ]:
single_email = "Dear friend=2CMy name is Barrister John Mensah=2C 41 years Ghanaianmarried with Five Children=2E I work as anadministrative Secretary=2FAttorney with GLOBALSECURITY COMPANY =28ltd=29 Accra Ghana=2E I joined theservices of this company in 1991=2EI got the information concerning you and your companyfrom our Chamber of Commerce here and after dueconsultation=2C I decided to contact you believing thatby the grace of God that you will be mypartner in this business=2E I have worked with thiscompany for over 12 years=2C within this period of timeI watched with meticulous precision onhow African Head of State and government functionarieshave been moving huge sums of money =28Dollars=2C PoundSterling=2C French Franc=2C Cash=29with the aid of theirforeign partners abroad=2EThey bring in these consignment of money cash andsecretly declare the content as jewelries=2C Gold=2CPrecious Stones=2C Family Treasure=2C Documents=2Cetc=2EMobutu Sese Seko of Zaire=2C Fodey Sankoy of SierraLeone=2C FelixHouphet Buigny of Ivory Coast=2C Konnan Bedeof Ivory Coast=2C etc=2EThese people have at one time or the otherdepositedconsignments withus and their foreignpartners=2C relatives and Next of Kin are Claiming mostof these consignment and a lot more are laying hereunclaimed for as much as 7 years=2C as no availableinformation to anybody except the depositors=2C and mostof them are dead=2E Since the inception of the year2003=2C GLOBAL SECURITY COMPANY LTD Management changedthe procedure of claiming of consignments=2EAs soon as you are able to produce all the secretinformation as contained in the secret file of thecompany as regard the consignment=2C it will be releasedto you upon demand=2E From our records=2C morethan 10 consignments belonging to these people hasbeen claimed in the past six months and this is why Iam seeking for your co-operation and assistance=2EMr=2E Francis Carden is a Zimbabwean=2C and he work withone of the top ministers of President Robert Mugabewho was in charge of the campaigning for re-electingMr=2E Mugabe into office in their last election=2E Hedeposited this consignment in 1999 after abscondingwith the money 35=2E5 million United State Dollars=28$35=2E5m USD=29=2EUnfortunately=2C Mr=2E Francis Carden died in Kenyanaircraft that killed all on board in 2000=2ESince his death=2C no one knows about this transactionwith Global Security Company Ltd except me who actedas company attorney when he deposited the consignment=2EI have finished every arrangement for you to come andclaim the consignment as the next of Kin to Mr=2EFrancis Carden=2EThe procedure is very simple=3A - Apply officially tothe Director of operation Global security company ptyAccra - Ghana=2C for the release of consignmentNo=2EGSS=2F15031964=2EThey will demand for some documents and the securitycode of the consignment=2C you will call on me to supplyyou with every detailed Information as regard theconsignment=2EAs soon as they have confirmed it to be correct=2C thecompany will invite you for the collection=2ENobody will ever know that I am involved in the deal=2C I suggest upon conclusion=2C we share it on the Ratioof 50%-40%and 10% will go for humanitarian service iemotherless babies home=2C more also you will make anarrangement on investments that I will invest my moneyin your country=2EI assure you that this business is very secure andrisk free=2EPlease=2C if you are not interested on this proposal=2Ckindly let me know=2EThanksBarrister John Mensah"

# Process the single email
processed_email_tensor = process_single_text(single_email, vocab, max_sequence_length=350, unk_index=vocab["<unk>"])



processed_email_tensor = processed_email_tensor.unsqueeze(0) 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processed_email_tensor = processed_email_tensor.to(device)


model = loaded_model.to(device)


model.eval()

# 6. Make a prediction
with torch.no_grad(): 
    logits = model(processed_email_tensor)

# 7. Apply softmax to get probabilities 
probabilities = torch.softmax(logits, dim=1)

label = torch.argmax(probabilities, dim=1).item()
print(label)

2


In [ ]:
single_email = "REMITTANCE OF $15 MILLION U.S.A DOLLARS         =                          CONFIDENTIAL IS THE CASE Compliments Of The Season 2007,    Forgive my indignation if this message c=omes to you as a surprise and may offend your personality for contacting yo=u without your prior consent and writing through this channel.I got your co=ntact from the proffesional data base found in the internet Yahoo tourist s=earch.When i was searching For a foreign reliable partner.I assured of your capability and reliability to champion this businees oppo=rtunity.  After series of prayers/fasting.i was divinely directed to contac=t you among other names found in the data base Yahoo tourist search.I belie=ve that God has a way of helping who is in need.I am (MUSTAFA BRUNU  Esq),the Head of file Department in BANK OF AFRICA (BO=A). In my department we discovered an abandoned sum of $ 15 million U.S.A d=ollars ( fifteen million U.S.A dollars) . In an account that belongs to one= of our foreign customer who died along with his entire family on (July 21,= 2003) in a plane crash. Since we got information about his death, we have =been expecting his next of kin to come over and claim his money because we =cannot =B5 release it unless somebody applies for it as next of kin or rela=tive on to the deceased as indicated in our banking guidelines,But unfortunately we learnt that all his supposed next of kin or relation d=ied alongside with him at the plane crash leaving nobody behind for the cla=im. It is therefore upon this discovery that I and one official in my depar=tment now decided to make this businness proposal to you and release the mo=ney to you as the next of kin or relation to the deceased for safety and su=bsequent disbursement since nobody is coming for it and we don=92t want thi=s money to go into the bank treasury as unclaimed bill.The Banking law and guideline here stipulates that if such money remained u=nclamed after five years, the money will be transfered into the Bank treasu=ry as unclaimed fund. The request of foreigner as next of kin in this busin=ess is occasioned by the fact that the customer was a foreigner,and a Burki=nabe cannot stand as next of kin to a foreigner. We agree that 30 % of this= money will be for you as foreign partner,in respect to the provision of a =foreign account, 10 % will be set aside for expenses incured during the bus=iness and 60 % would be for me and my woman colleague. There after i will v=isit your country for disbursement according to the percentages indicated.Therefore to enable the immediate transfer of this fund  as arranged, you m=ust apply first to the bank as relations or next of kin of the deceased ind=icating your bank name, your bank account number, your private telephone an=d fax number for easy and effective communication and location where the mo=ney will be remitted.Upon receipt of your reply, I will send to you by fax =or email the text of the application I will not fail to bring to your notic=e that this transaction is hitch free and that you should not entertain any= atom of fear as all required arrangements have been made for the transfer.You should contact me on my number +226-76 54 93 42. immediately as soon as= you receive this letter. Trusting to hear from you immediately.Your=92s fa=ithfully,MUSTAFA BRUNU  (Esq.)FROM (B.O.A) OUAGADOUGOU BURKINA-FASO.GOD BLESSINGS,PROTECTIONS AND GUIDIAN=CE TO YOUR ENTIRE FAMILY.POST SCRITUM:You have to keep everything secret as= to enable the transfer to move very smoothly in to the account you will pr=ove to the bank.As you finished reading this letter call me immediately so =that we discuss very well over this business._________________________________________________________________Play free games, earn tickets, get cool prizes! Join Live Search Club.=A0http://club.live.com/home.aspx?icid=3DCLUB_wlmailtextlink="

# Process the single email
processed_email_tensor = process_single_text(single_email, vocab, max_sequence_length=350, unk_index=vocab["<unk>"])



processed_email_tensor = processed_email_tensor.unsqueeze(0)  


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processed_email_tensor = processed_email_tensor.to(device)


model = loaded_model.to(device)


model.eval()

# 6. Make a prediction
with torch.no_grad(): 
    logits = model(processed_email_tensor)

# 7. Apply softmax to get probabilities
probabilities = torch.softmax(logits, dim=1)

label = torch.argmax(probabilities, dim=1).item()
print(label)

1


safe email testing

In [ ]:
single_email = 'J - can u print for hrc? Latest version of doha speech. I\'m sure others will have edits but wanted her to have copy now.'

# Process the single email
processed_email_tensor = process_single_text(single_email, vocab, max_sequence_length=350, unk_index=vocab["<unk>"])



processed_email_tensor = processed_email_tensor.unsqueeze(0)  


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processed_email_tensor = processed_email_tensor.to(device)


model = loaded_model.to(device)


model.eval()

# 6. Make a prediction
with torch.no_grad():  
    logits = model(processed_email_tensor)

# 7. Apply softmax to get probabilities 
probabilities = torch.softmax(logits, dim=1)

label = torch.argmax(probabilities, dim=1).item()
print(label)

0


In [ ]:
single_email = 'H <hrod17@clintonemail.com>Friday March 11 2011 1:36 PMHuma AbedinFw: H: Latest: How Syria is aiding Qaddafi and more... Sidhrc memo syria aiding libya 030311.docxPis print.'

# Process the single email
processed_email_tensor = process_single_text(single_email, vocab, max_sequence_length=350, unk_index=vocab["<unk>"])



processed_email_tensor = processed_email_tensor.unsqueeze(0)  


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processed_email_tensor = processed_email_tensor.to(device)


model = loaded_model.to(device)


model.eval()

# 6. Make a prediction
with torch.no_grad():  # No need to track gradients during inference
    logits = model(processed_email_tensor)

# 7. Apply softmax to get probabilities
probabilities = torch.softmax(logits, dim=1)

label = torch.argmax(probabilities, dim=1).item()
print(label)

0


In [ ]:
single_email = 'Well most of the weekend planes are the little planes. Calling mario now to see..'

# Process the single email
processed_email_tensor = process_single_text(single_email, vocab, max_sequence_length=350, unk_index=vocab["<unk>"])



processed_email_tensor = processed_email_tensor.unsqueeze(0) 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processed_email_tensor = processed_email_tensor.to(device)


model = loaded_model.to(device)


model.eval()

# 6. Make a prediction
with torch.no_grad(): 
    logits = model(processed_email_tensor)

# 7. Apply softmax to get probabilities 
probabilities = torch.softmax(logits, dim=1)

label = torch.argmax(probabilities, dim=1).item()
print(label)

0


In [ ]:
single_email = 'Dan FeldmanImportance: HighI\'m involved in preparations for the Holbrooke memorial service at the Kennedy Center on Jan.14 and I need to get your guidance/decision on your own role. I\'m reachable through Ops oron my cell although I\'ll be out-of-pocket between 5 and 7 thisafternoon/evening. Regards StrobeStrobe TalbottPresident The Brookings Institution1775 Massachusetts Avenue NWWashington DC 20036702 797 6700'

# Process the single email
processed_email_tensor = process_single_text(single_email, vocab, max_sequence_length=350, unk_index=vocab["<unk>"])



processed_email_tensor = processed_email_tensor.unsqueeze(0) 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processed_email_tensor = processed_email_tensor.to(device)


model = loaded_model.to(device)


model.eval()

# 6. Make a prediction
with torch.no_grad():  
    logits = model(processed_email_tensor)

# 7. Apply softmax to get probabilities 
probabilities = torch.softmax(logits, dim=1)

label = torch.argmax(probabilities, dim=1).item()
print(label)

0


In [ ]:
single_email = 'Just wanted to tell you that speech yesterday was terrific. Important to say that never will be perfect security; thatdiplomats can\'t operate in bunkers; that the world is a dangerous place. Thought NYT article this morning helpful aswell We have understandably been more on defense but love the offense. Thank you If any of us can help reinforce letus knowU.S. Department of StateCase No. F-2015-04841Doc No. C05739700Date: 05/13/2015STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045701'

# Process the single email
processed_email_tensor = process_single_text(single_email, vocab, max_sequence_length=350, unk_index=vocab["<unk>"])



processed_email_tensor = processed_email_tensor.unsqueeze(0) 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processed_email_tensor = processed_email_tensor.to(device)


model = loaded_model.to(device)


model.eval()

# 6. Make a prediction
with torch.no_grad():  
    logits = model(processed_email_tensor)

# 7. Apply softmax to get probabilities 
probabilities = torch.softmax(logits, dim=1)

label = torch.argmax(probabilities, dim=1).item()
print(label)

0
